<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Job%20Shop%20Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Shop Scheduling Problem

Matematical model from: Bülbül, K., & Kaminsky, P. (2013). A linear programming-based method for job shop scheduling. *Journal of Scheduling*, 16, 161-183.

## Notation

\begin{array}{ll}
m & \text{Number of machines}\\
n & \text{Number of jobs}\\
M_j  & \text{Ordered sequence of machines for job } j\\
J_i  & \text{Set of operations to be processed on machine } i\\
p_{ij} & \text{Processing time for job $i$ on machine $i$} \\
r_{j} & \text{Ready time for job } j\\
h_{ij} & \text{Holding cost per unit time for job $j$ while it is waiting in the queue before machine } i\\
w_{ij} & \text{Decision variable (integer) denoting the time job $j$ spends in the queue before machine $i$}\\
C_{ij} & \text{Decision variable (integer) denoting the time at which job $j$ finishes processing on machine $i$}\\
\end{array}
<br>

## Mathematical Model

$
\begin{align}
  \begin{array}{llll}
    &\textbf{Objective} & \\
    & \min \sum\limits_{j=1}^{n} \sum\limits_{i=1}^{m_j} h_{ij} \cdot w_{ij} & &~~~  (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & C_{1j} - w_{1j} = r_j + p_{1j}&,~ \forall~ j=1,...,n  &~~~ (2) \\
    & C_{i-1,j} - C_{ij} + w_{ij} = -p_{ij}&,~ \forall~ j=1,...,n, i=2,...,m_j &~~~ (3) \\
    & C_{ik} - C_{ij} \ge p_{ik}&,~ \forall~ i=1,...,n, i,k \in J_i &~~~ (4a) \\
    & C_{ij} - C_{ik} \ge p_{ij}&,~ \forall~ i=1,...,n, i,k \in J_i &~~~ (4b) \\
    & C_{ij}, w_{ij} \ge 0 &,~ \forall~ j=1,...,n, i=2,...,m_j &~~~ (5) \\
  \end{array}
\end{align}
$
<br><br>
**Important:** Either (4a) OR (4b) not both!

## Python implementation

### Exemplary data

In [3]:
# number of machines
m = 4

# number of jobs
n = 2

# Ordered sequence of machines for a job
M = {1: [1, 3, 4],
     2: [1, 4, 2]}

# Set of operations on a machine
J = {1: [1, 2],
     2: [2],
     3: [1],
     4: [1, 2]}

# processing times
p = {1: {1: 6, 3: 4, 4: 1},
     2: {1: 2, 2: 3, 4: 9}}

# ready times equals zero for jobs
r = {1: 0, 2: 0}
r = {j+1: 0 for j in range(n)}

# waiting costs equal one for all machines
h = {i+1: {j+1: 1 for j in range(m)} for i in range(n)}

{1: {1: 1, 2: 1, 3: 1, 4: 1}, 2: {1: 1, 2: 1, 3: 1, 4: 1}}
{1: {1: 1, 2: 1, 3: 1, 4: 1}, 2: {1: 1, 2: 1, 3: 1, 4: 1}}


### Packages

In [ ]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Model

In [ ]:
# model
model = pulp.LpProblem(name='Orienteering_Problem', sense=pulp.constants.LpMaximize)

# decision variables
x = pulp.LpVariable.dicts(name='x', indices=(nodes, nodes), cat='Binary')
u = pulp.LpVariable.dicts(name='u', indices=nodes[1:],
                          lowBound=2, upBound=len(nodes),
                          cat='Integer')

# objective
model += pulp.lpSum(S[i]*x[i][j] for i in nodes[1:-1] for j in nodes[1:]), '(0)'

# (1a)
model += pulp.lpSum(x[nodes[0]][j] for j in nodes[1:]) == 1, '(1a)'

# (1b)
model += pulp.lpSum(x[i][nodes[-1]] for i in nodes[:-1]) == 1, '(1b)'

for k in nodes[1:-1]:
  # (2a)
  model += pulp.lpSum(x[i][k] for i in nodes[:-1]) <= 1, '(2a):' + k
  # (2b)
  model += pulp.lpSum(x[i][k] for i in nodes[:-1]) <= pulp.lpSum(x[k][j] for j in nodes[1:]), '(2b):' + k

# (3)
model += pulp.lpSum(t[i][j] * x[i][j] for i in nodes[:-1] for j in nodes[1:]) <= T_max, '(3)'

# (4) -> see initialization of decision variable u

# (5)
for i in nodes[1:]:
  for j in nodes[1:]:
    model += u[i] - u[j] + 1 <= (len(nodes) - 1) * (1 - x[i][j]), '(5):' + i + j

# (6) -> see initialization of decision variable x



### Solution

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get tour
print('Tour: ')
t_sum = 0
i = nodes[0]
while i != nodes[-1]:
  for j in nodes:
    try:
      if x[i][j].varValue > 0:
        print(' ' + str(x[i][j]) + ': ' + str(x[i][j].varValue) + ', t_ij=' + str(t[i][j]) + ', S_j=' + str(S[j]))
        t_sum += t[i][j]
        i = j
        break
    except:
      pass
print(' total time: ' + str(t_sum))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
print('')
print('u-variable: ')
for i in nodes[1:]:
  print(' ' + str(u[i]) + ': ' + str(u[i].varValue))

Status: Optimal
Objective value: 96.0
Tour: 
 x_node_0_node_2: 1.0, t_ij=17, S_j=73
 x_node_2_node_1: 1.0, t_ij=29, S_j=13
 x_node_1_node_3: 1.0, t_ij=21, S_j=10
 x_node_3_node_5: 1.0, t_ij=14, S_j=6
 total time: 81

u-variable: 
 u_node_1: 3.0
 u_node_2: 2.0
 u_node_3: 4.0
 u_node_4: 2.0
 u_node_5: 6.0
